#<center>**Parcial 1**</center>
<center><small><b>Presentado por:<br> Santiago Pérez González </small></b></center> 

---
<center>En este archivo se presenta la realización del parcial de Aprendizaje de Máquina <br> <b> Profesor: Camilo Rodríguez </b><br> Universidad Sergio Arboleda</center>

Crear una clase KNN que represente el modelo del vecino más cercano para clasificación:
* El constructor debe recibir el número de vecinos
* Debe tener un método fit, que reciba X, y
* Debe tener un método predict que prediga a que clase pertenece una muestra X
* Debe tener un métoto predict_proba que reciba X y devuelva las probabilidades de que pertenezca a cada una de las clases.

Se crea una clase base para punto 1

In [23]:
class Model:
    def __init__(self):
        pass
    def fit(X,y):
        pass
    def predict(X):
        pass
    def predict_proba(X):
        pass

##Punto 1

Desarrollo de la clase para KNN

In [24]:
class KNN(Model):
    def __init__(self,k):
        """
        Constructor of KNN class
        k - that is the neighbor number
        """
        super().__init__()
        self.neighbours=k
        self.X_train=None
        self.y=None
    def fit(self,X,y):
        """
        Fit the model with KNN algorithm
        X - features
        y - target
        """
        self.X_train=X
        self.y=y
    def predict(self,X):
        """
        Predict the labels of the data
        X - features
        """
        prediction=[]
        for x in X:
            distances=[]
            for x_train in self.X_train:
                distances.append((x_train,self.distance(x,x_train)))
            distances.sort(key=lambda x:x[1])
            k_nearest=distances[:self.neighbours]
            k_nearest_labels=[]
            for k in k_nearest:
                k_nearest_labels.append(self.y[self.X_train.index(k[0])])
            prediction.append(self.majority(k_nearest_labels))
        return prediction
    def predict_proba(self,X):
        """
        Predict the probabilities of the data
        X - features
        Returns the probabilities of the data (for each label)
        """
        for x in X:
            distances=[]
            for x_train in self.X_train:
                distances.append(self.distance(x,x_train))
            distances.sort()
            k_nearest=distances[:self.neighbours]
            k_nearest_labels=[]
            for k in k_nearest:
                k_nearest_labels.append(self.y[self.X_train[k]])
            self.y.append(self.probability(k_nearest_labels))
        return (self.y,[1-self.y[i] for i in range(len(self.y))])
    def distance(self,x,x_train=None):
        """
        Calculate the distance between two data
        x - data
        x_train - data
        """
        return sum([(x[i]-x_train[i])**2 for i in range(len(x_train))])
    def majority(self,k_nearest_labels):
        """
        Calculate the majority of the labels
        k_nearest_labels - labels of the k nearest data
        """
        return max(set(k_nearest_labels),key=k_nearest_labels.count)
    def probability(self,k_nearest_labels):
        """
        Calculate the probabilities of the labels
        k_nearest_labels - labels of the k nearest data
        """
        return k_nearest_labels.count(self.majority(k_nearest_labels))/self.neighbours

Ejecución

In [25]:
import numpy as np
x1 = np.array([2.3, 2.1, 3.7, 6.1, 6, 9.2, 11.0])
x2 = np.array([1.3, 5.1, 0.7, 9.1, 4, 3.3, 5.0])
X_train=[[x,y] for x,y in zip(x1,x2)]

y = np.array([0, 1, 1, 0, 1, 0, 0])
knn = KNN(3)
knn.fit(X_train,y)
print(knn.predict([[9,5],[3,3]]))

[0, 1]


#Punto 3

In [40]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

In [27]:
df = pd.read_csv("https://raw.githubusercontent.com/Pasta43/MachineLearning/main/first-term/datasets/heart.csv")

In [28]:
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,59,1,1,140,221,0,1,164,1,0.0,2,0,2,1
1021,60,1,0,125,258,0,0,141,1,2.8,1,1,3,0
1022,47,1,0,110,275,0,0,118,1,1.0,1,1,2,0
1023,50,0,0,110,254,0,0,159,0,0.0,2,0,2,1


In [29]:
train, test = train_test_split(df, test_size=0.3, random_state=42)

In [30]:
def metric_summary(results):
  for key in results.keys():
    print(f"{key}: {results[key].mean()} +- {results[key].std()}" )

In [48]:
def scale_data(df):
  scaler = StandardScaler()
  scaler.fit(df)
  scaled=scaler.transform(df)
  data_scaled= pd.DataFrame(data=scaled, columns=df.columns)
  return data_scaled

In [54]:
#Train data
X_train = train.drop('target', axis=1)
y_train = train['target']

#Scaled data
scaled= scale_data(train)
X_scaled =scaled.drop('target', axis=1)
y_scaled = train['target']

In [55]:
predictors=[[DecisionTreeClassifier()],[LogisticRegression()],[KNeighborsClassifier(),1],[KNeighborsClassifier(),3],[KNeighborsClassifier(),5],[KNeighborsClassifier(),7]]

In [58]:
for predictor in predictors:
  X_to_train=None
  y_to_train=None
  message=""
  clf = None
  if (type(predictor[0])==type(DecisionTreeClassifier())):
    message="Para árbol de clasificación"
    clf=predictor[0]
    X_to_train=X_train
    y_to_train=y_train
  elif (type(predictor[0])==type(LogisticRegression())):
    message="Para regresión logística"
    clf=predictor[0]
    X_to_train=X_scaled
    y_to_train=y_scaled
  elif (type(predictor[0])==type(KNeighborsClassifier())):
    message="Para KNN con "+str(predictor[1]) +" vecinos"
    clf=predictor[0]
    clf.n_neighbors=predictor[1]
    X_to_train=X_scaled
    y_to_train=y_scaled
  results = cross_validate(clf, X_to_train, y_to_train,
               cv=10, scoring=['accuracy', 'precision', 'recall', 'f1'])
  print(message)
  metric_summary(results)

Para árbol de clasificación
fit_time: 0.004210114479064941 +- 0.001113247140206847
score_time: 0.004598784446716309 +- 0.00040638786425280634
test_accuracy: 0.9804186228482005 +- 0.020046318047188916
test_precision: 0.9775174725431464 +- 0.03358807116510971
test_recall: 0.9866998577524895 +- 0.01797349506497893
test_f1: 0.9816664918622335 +- 0.01856797971632296
Para regresión logística
fit_time: 0.007432079315185547 +- 0.0027730052628599333
score_time: 0.005407547950744629 +- 0.0015777892167168803
test_accuracy: 0.8561815336463224 +- 0.04737068780513644
test_precision: 0.8368690094429585 +- 0.06025973752785159
test_recall: 0.9096728307254622 +- 0.0342341028881737
test_f1: 0.8702748709170786 +- 0.03791373155535019
Para KNN con 1 vecinos
fit_time: 0.0031867027282714844 +- 0.00023293312154029383
score_time: 0.007785820960998535 +- 0.0017305433226137879
test_accuracy: 0.9776408450704224 +- 0.025948886296008618
test_precision: 0.9707750379701598 +- 0.04278663074174251
test_recall: 0.9893314

Teniendo en cuenta lo anterior, el mejor puntaje lo obtuvo el árbol de clasificación con:
* *accuracy* de  0.9804186228482005,
* *precision* de 0.9775174725431464
* *recall* de 0.9866998577524895 y
* $f_{1measure}$ de  0.9816664918622335

In [59]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

In [60]:
clf=DecisionTreeClassifier()

In [61]:
clf.fit(X_train,y_train)

DecisionTreeClassifier()

In [62]:
#test data
X_test = test.drop('target', axis=1)
y_test = test['target']

In [63]:
y_predict=clf.predict(X_test)

In [65]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_predict)

array([[159,   0],
       [  9, 140]])

In [64]:
print("Accuracy:", accuracy_score(y_test,y_predict),
      "Precision:",precision_score(y_test,y_predict),
      "Recall:",recall_score(y_test,y_predict),
      "f1:",f1_score(y_test,y_predict))

Accuracy: 0.9707792207792207 Precision: 1.0 Recall: 0.9395973154362416 f1: 0.9688581314878892


## Análisis de resultados

Al realizar el entrenamiento se corría el riesgo de obtener overfitting, pero el árbol de clasificación mostró un alto desempeño con los datos de prueba dando como resultado los puntajes de 

* Accuracy: 0.9707792207792207
* Precision: 1.0 
* Recall: 0.9395973154362416 
* f1: 0.9688581314878892

Indicando que parece que se tiene buen modelo con la técnica del árbol de clasificación para estos datos dado el contexto del problema es decir (se obtuvo una precisión alta y un recall bajo, que es preferido para la empresa prestadora de salud en no gastar tiempo y dinero en procedimientos médicos en alguien que no tenga riesgo de padecer un ataque al corazón):



Añadendo a esto, analizando la matriz de confusión, se obtiene:
* De 168 personas que tienen riesgo de padecer un ataque al corazón, detectó 159
* El modelo no tuvo error al detectar a alguien que no tenía riesgo de ataque al corazón, es decir, de 140 personas sin riesgo de padecer ataques al corazón, no determinó que ninguna de estas tuviera dicho riesgo.
* De 9 personas que tenían riesgo de padecer un ataque al corazón, fueron predichas en que no tenían dicho riesgo